In [67]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import math
import csv
import os
import cv2


In [68]:
lat_lon_file_path='/Users/moctader/Thesis_code/lat_lon/results.csv'
feature_vector_path='/Users/moctader/Thesis_code/lat_lon/feature_vector.csv'
csv_file_path = '/Users/moctader/Thesis_code/lat_lon/results.csv'
tiles_template = "/Users/moctader/Thesis/{t}/{z}/{x}/{y}.png"
combined_csv_path = '/Users/moctader/Thesis_code/lat_lon/combined_results.csv'
points = gpd.read_file("/Users/moctader/Thesis_code/GTK_ASsoil_obs.csv")
t_values = glob.glob('/Users/moctader/Thesis/*')
base_directory = "/Users/moctader/Thesis_code/output20/"


In [69]:
points.POINT_X = points.POINT_X.astype("float")
points.POINT_Y = points.POINT_Y.astype("float")
zoom_level = 10



In [70]:
#samples
samples = gpd.GeoDataFrame(
    points.CLASS, crs="EPSG:3067", geometry=gpd.points_from_xy(points.POINT_X, points.POINT_Y)
).to_crs("WGS84")

tile_list = [(point.x, point.y) for point in samples['geometry']]


In [71]:
samples["i"] = samples.index
samples["filenames"] = samples.apply(lambda row: f"{row['CLASS']}/image_{row['i']}", axis=1)



In [72]:
def get_lat_from_row(p):
    lon, lat = p.geometry.x, p.geometry.y
    return lat

def get_lon_from_one_column(geometry):
    # geometry is a column
    lon, lat = geometry.x, geometry.y
    return lon

In [73]:
samples["lat"] = samples.apply(lambda row: get_lat_from_row(row), axis=1)

In [74]:
samples["lon"] = samples["geometry"].map(get_lon_from_one_column)


In [75]:
# load data 
def load_data(filename, directory):
    path = directory + "/" + filename + ".png"
    image_array = np.array(Image.open(path))

    if len(image_array.shape) == 3 and image_array.shape[2] == 3:
        return image_array[:, :, :3]
    else:
        return image_array[:, :, 0:1]

    
    

In [76]:
# add all the images
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    samples[single_file] = samples["filenames"].map(
        lambda name, directory=os.path.join(base_directory, single_file): load_data(name, directory)
    )


In [77]:
#create label
def label_rows(row):
    if 'ASS' == row['CLASS']:
        return 1
    else:
        return 0

samples['label'] = samples.apply(label_rows, axis=1)


In [78]:

# All channels togather to from combine channel

combined_images_list = []

# Iterate over rows of the DataFrame
for index, row in samples.iterrows():
    # Initialize an empty list to store individual layers
    layers_list = []

    # Iterate over the selected columns
    for single_file in files:
        layer = (row[single_file])
        layers_list.append(layer)

    # Concatenate the layers along the third dimension to create a multi-channel image
    combined_image = np.concatenate(layers_list, axis=-1)
    
    # Append the combined image to the list
    combined_images_list.append(combined_image)

# Add the combined images as a new column in the DataFrame

samples["combined_channels"] = combined_images_list



In [79]:
print(samples.iloc[5623:5627])

        CLASS                   geometry     i           filenames        lat  \
5623  non-ASS  POINT (25.48415 65.41564)  5623  non-ASS/image_5623  65.415640   
5624  non-ASS  POINT (25.59713 65.42222)  5624  non-ASS/image_5624  65.422220   
5625      ASS  POINT (25.41993 65.43876)  5625      ASS/image_5625  65.438758   
5626      ASS  POINT (25.44373 65.43709)  5626      ASS/image_5626  65.437087   

            lon                                 elev_10m_hillshade  \
5623  25.484148  [[[198, 198, 198], [198, 198, 198], [194, 194,...   
5624  25.597127  [[[198, 198, 198], [198, 198, 198], [198, 198,...   
5625  25.419933  [[[196, 196, 196], [196, 196, 196], [196, 196,...   
5626  25.443733  [[[196, 196, 196], [196, 196, 196], [196, 196,...   

                                                landsat  \
5623  [[[125, 59, 106], [127, 62, 106], [86, 64, 104...   
5624  [[[228, 181, 148], [183, 200, 142], [215, 223,...   
5625  [[[1, 2, 44], [2, 3, 45], [1, 2, 44], [1, 2, 4...   
5626  [

In [80]:
samples.to_pickle("/Users/moctader/Thesis_code/pickle/samples.pkl")

In [81]:





# This is not part of dataframe 
# Check the images chanels

base_directory = "/Users/moctader/Thesis_code/output20/"

# Get a list of file_names in the base directory
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    #mage_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/non-ASS/image_5786.png'
    image_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/ASS/image_0.png'
    image = Image.open(image_file_path)

    # Get the bands (channels) of the image
    bands = image.getbands()
    
    # Determine the number of channels
    num_channels = len(bands)

    # Print the result
    print(f"The image at {image_file_path} has {num_channels} channel(s): {bands}")

The image at /Users/moctader/Thesis_code/output20/elev_10m_hillshade/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/landsat/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m_slope/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_real_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/corine_land_cover/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_apparent_resistivity/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_imaginary_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/TPI/ASS/image